In [1]:
#task 1
import keras
import numpy as np  
import pandas as pd
import util_mnist_reader
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

losstrack = []

x_train, y_train = util_mnist_reader.load_mnist('../data/fashion', kind='train')
x_test, y_test = util_mnist_reader.load_mnist('../data/fashion', kind='t10k')

# for feature in x_train.columns:
#     max_value = df[feature].max()
#     min_value = df[feature].min()
#     x[feature] = (df[feature] - min_value) / (max_value - min_value)  #NORMALIZATION
   

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 16)  
#SPLITTING INTO 80% TRAIN 10% VALIDATION

#normalization
x_train=x_train.astype('float32')
x_train = x_train/255
x_test=x_test.astype('float32')
x_test = x_test/255
x_val=x_val.astype('float32')
x_val = x_val/255

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


# y_train = y_train.reshape( y_train.shape[0],1)
# y_test = y_test.reshape( y_test.shape[0],1)
# y_val = y_val.reshape( y_val.shape[0],1)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('xval shape',x_val.shape)
print('yval shape',y_val.shape)


#INITIALISATION
epochs = 50
learningrate = 0.01
hiddenlayers=1024

# def loss(a,y):        
#     loss1=np.sum(np.multiply(y_train,np.log(p2)))
#     m=y_train.shape[0]
#     loss= -(1/m) * loss1
#     return loss

def sigmoid(z):
    return 1 / (1 + np.exp(-z))
losstrack = [] #ARRAY OF LOSS

def softmax(z1):
    return np.exp(z1) / np.sum(np.exp(z1))


w1 = np.random.randn(x_train.shape[1], hiddenlayers)
b1 = np.zeros(hiddenlayers)
w2 = np.random.randn(hiddenlayers, 10)
b2 = np.zeros(10)

for epoch in range(epochs):
    #forward prop
    z1 = np.dot( x_train,w1) + b1
    p1 = sigmoid(z1)
    z2 = np.dot( p1,w2) + b2
    p2 = softmax(z2)
   
    loss1=np.sum(np.multiply(y_train,np.log(p2)))
    m=y_train.shape[0]
    loss= -(1/m) * loss1
    losstrack.append(np.squeeze(loss))
   
    #backward prop
   
    result1=p2 - y_train
    val=np.dot(result1,w2.T)
    result2=np.multiply(p1 , (1-p1))
    result3=np.multiply(val,result2)
   
    dw2=(1./m) * np.dot(result1.T,p1)
    db2=(1./m)*np.sum(result1)
   
    dw1=(1./m) * np.dot(x_train.T,result3)
    db1=(1./m)*np.sum(result3)
   

    w1 = w1 - learningrate * dw1
    b1 = b1 - learningrate * db1
    w2 = w2 - learningrate * dw2.T
    b2 = b2 - learningrate * db2
   
    print("epoch")
    print(epoch)
    print("loss")
    print(loss)
   



plt.figure(1) #graph : training set vs epochs
plt.title("TRAINING SET")
plt.xlabel("EPOC")
plt.ylabel("COST")
plt.plot(losstrack,label='TRAINING SET')
plt.legend()
z1 = np.dot( x_test,w1) + b1
p1 = sigmoid(z1)
z2 = np.dot( p1,w2) + b2
p2 = softmax(z2)

predictions = np.argmax(p2, axis=1)
labels = np.argmax(y_test, axis=1)

print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))
print("Accuracy")
print(accuracy_score(predictions,labels ))


In [2]:
# Task2

import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
import scipy
import matplotlib.pyplot as plt
import util_mnist_reader
import keras
from numpy import loadtxt
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation,Flatten
x_train, y_train = util_mnist_reader.load_mnist('../data/fashion', kind='train')
x_test, y_test = util_mnist_reader.load_mnist('../data/fashion', kind='t10k')



y_train = keras.utils.to_categorical(y_train1)

y_test = keras.utils.to_categorical(y_test1)
x_train=x_train.reshape(x_train.shape[0],28,28)
x_test=x_test.reshape(x_test.shape[0],28,28)
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)


model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128, activation='relu', input_shape=(28,28)))

model.add(Dense(128, activation='tanh'))


model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

fit1=model.fit(x_train, y_train,validation_data=(x_test,y_test),epochs=100,batch_size=512)
plt.plot(fit1.history['loss'])
plt.plot(fit1.history['val_loss'])
plt.title("loss vs epoch")
plt.plot(loss,label='TRAINING SET')
plt.plot(val_loss,label='VALIDATION SET')
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()
# score = model.evaluate(x_test, y_test, batch_size=128)
# print(score)

#Model evaluation
y_pred = model.predict(x_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

#Confusion matrix
cm = confusion_matrix(test,pred)
print("Confusion matrix is:")
print(cm)


In [ ]:
#task 3
import matplotlib.pyplot as plt
import keras

import util_mnist_reader
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
batch_size = 128
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

x_train, y_train = util_mnist_reader.load_mnist('../data/fashion', kind='train')
x_test, y_test = util_mnist_reader.load_mnist('../data/fashion', kind='t10k')




x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

fit1=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
plt.plot(fit1.history['loss'])
plt.plot(fit1.history['val_loss'])
plt.plot(loss,label='TRAINING SET')
plt.plot(val_loss,label='VALIDATION SET')
plt.title("loss vs epoch")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()
# score = model.evaluate(x_test, y_test, verbose=0)

# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
#Model evaluation
y_pred = model.predict(x_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

#Confusion matrix
cm = confusion_matrix(test,pred)
print("Confusion matrix is:")
print(cm)

